In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
df = pd.read_csv("data_after_cleansing_normalized.csv")

In [6]:
df.head()

,Tweet,HS,normalized_tweet,clean_tweet
0,- disaat semua cowok berusaha melacak perhatia...,1,- di saat semua pria berusaha melacak perhatia...,di saat semua pria berusaha melacak perhatian...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,RT USER: USER siapa telat memberi tau elu?eda...,rt user user siapa telat memberi tau eluedan s...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,"41. Kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya ...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,user user aku itu akunnku tau matamu sipit tap...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,USER USER Kaum kecebong kafir sudah kelihatan ...,user user kaum kecebong kafir sudah kelihatan ...


In [8]:
X = df['clean_tweet'] #Text
y = df['HS'] #Label
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Tokenisasi teks
max_words = 5000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [10]:
# Mengonversi teks ke dalam sequence
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [11]:
# Padding sequence agar memiliki panjang yang sama
maxlen = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post')

In [24]:
# Membangun model LSTM
embedding_dim = 64
lstm_units = 64

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
# Training model
epochs = 15
batch_size = 256

history = model.fit(X_train_pad, y_train, 
                    epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test_pad, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/15
58/58 [==============================] - 19s 290ms/step - loss: 0.6759 - accuracy: 0.5967 - val_loss: 0.6775 - val_accuracy: 0.5889
Epoch 2/15
58/58 [==============================] - 18s 303ms/step - loss: 0.6747 - accuracy: 0.5966 - val_loss: 0.6775 - val_accuracy: 0.5889
Epoch 3/15
58/58 [==============================] - 22s 383ms/step - loss: 0.6743 - accuracy: 0.5966 - val_loss: 0.6773 - val_accuracy: 0.5889
Epoch 4/15
58/58 [==============================] - 22s 376ms/step - loss: 0.6751 - accuracy: 0.5966 - val_loss: 0.6777 - val_accuracy: 0.5889
Epoch 5/15
58/58 [==============================] - 22s 373ms/step - loss: 0.6751 - accuracy: 0.5966 - val_loss: 0.6779 - val_accuracy: 0.5889
Epoch 6/15
58/58 [==============================] - 18s 310ms/step - loss: 0.6746 - accuracy: 0.5966 - val_loss: 0.6775 - val_accuracy: 0.5889


In [28]:
# Prediksi
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)  # Ambil kelas dengan probabilitas > 0.5

115/115 [==============================] - 1s 9ms/step


In [29]:
# Evaluasi model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy}')

115/115 [==============================] - 1s 10ms/step - loss: 0.6775 - accuracy: 0.5889
Test Accuracy: 0.5888587236404419
